In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 4bff579e-e92c-4091-91ab-7e55075e5782, 4, Finished, Available, Finished)

This cell sets Spark session settings to enable Verti-Parquet and Optimize on Write. More details about Verti-Parquet and Optimize on Write in tutorial document.

In [3]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.

spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

StatementMeta(, 4bff579e-e92c-4091-91ab-7e55075e5782, 5, Finished, Available, Finished)

In [12]:
# Load the dimension tables
dim_date =  spark.read.table("Gold.DimDate")
dim_time =  spark.read.table("Gold.DimTime")

print(dim_date)
print(dim_time)


StatementMeta(, 4bff579e-e92c-4091-91ab-7e55075e5782, 14, Finished, Available, Finished)

DataFrame[DateID: int, Date: date, Day: int, Month: int, Year: int, DayOfWeek: int, IsWeekend: string]
DataFrame[TimeID: int, Time: timestamp, Hour: int, Minute: int, Second: int, PeriodOfDay: string]


In [6]:

# Load the existing fact DataFrame
fact_df =  spark.read.table("Gold.fact_trip")  # Replace with the actual path to your fact table

StatementMeta(, 4bff579e-e92c-4091-91ab-7e55075e5782, 8, Finished, Available, Finished)

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, avg, date_format


# Join the fact DataFrame with the dimension tables
joined_df = fact_df \
    .join(dim_date, fact_df["DateID"] == dim_date["DateID"], "inner") \
    .join(dim_time, fact_df["TimeID"] == dim_time["TimeID"], "inner") 

print(joined_df)    

# Perform group by and aggregations
aggregated_df = joined_df.groupBy("Gold.DimDate.DateID") \
                         .agg(
                              count("*").alias("TotalTrips"),
                              sum("PassengerCount").alias("TotalPassengers"),
                              avg("TripDistance").alias("AverageTripDistance"),
                              sum("FareAmount").alias("TotalFareAmount"),
                              sum("TipAmount").alias("TotalTipAmount"),
                              sum("TotalAmount").alias("TotalTripAmount")
                         )

# Rename columns if necessary
renamed_df = aggregated_df.withColumnRenamed("DateID", "TripDate")

# Write the result as a Delta table
renamed_df.write.mode("overwrite").format("delta").save("Tables/fact_aggregated_trip")



StatementMeta(, 4bff579e-e92c-4091-91ab-7e55075e5782, 18, Finished, Available, Finished)

DataFrame[TripID: int, VendorID: int, DateID: int, TimeID: int, puLocationId: string, doLocationId: string, RateCodeID: int, StoreAndFwdFlag: string, PaymentType: int, TripType: int, PassengerCount: int, TripDistance: double, FareAmount: double, Extra: double, MtaTax: double, ImprovementSurcharge: double, TipAmount: double, TollsAmount: double, EhailFee: double, TotalAmount: double, DateID: int, Date: date, Day: int, Month: int, Year: int, DayOfWeek: int, IsWeekend: string, TimeID: int, Time: timestamp, Hour: int, Minute: int, Second: int, PeriodOfDay: string]
